# Census manifolds with rank-$1$ first homology

In [1]:
import twisted_l2
import itertools
import regina
import snappy

There are $127$ closed orientable manifolds in the census of hyperbolic $3$-manifolds:

In [2]:
len(snappy.OrientableClosedCensus(betti=1))

127

$86$ of them are fibered, the remaning $41$ are:

In [3]:
nonfibered_names = ["s528(-1,3)",  "s527(-5,1)",  "s644(-4,3)",  "s643(-5,1)",  "v2018(-4,1)",
     "v1436(-5,1)", "s750(4,3)",   "s749(5,1)",   "s789(-5,2)",  "v1539(5,2)",  "v2238(-5,1)",
     "v3209(1,2)",  "s828(-4,3)",  "v1695(5,1)",  "s862(7,1)",   "v2190(4,1)",  "v3209(-1,2)",
     "v2593(4,1)",  "v3209(3,2)",  "v3027(-3,1)", "v2896(-6,1)", "v2683(-6,1)", "v2796(4,1)",
     "v2797(-3,4)", "v2948(-6,1)", "v2794(-6,1)", "v3183(-3,2)", "v3145(3,2)",  "v3181(-3,2)",
     "v3036(3,2)",  "v3209(1,3)",  "v3269(4,1)",  "v3209(-3,2)", "v3209(2,3)",  "v3313(3,1)",
     "v3239(3,2)",  "v3209(5,2)",  "v3209(-1,3)", "v3209(4,3)",  "v3244(4,3)",  "v3243(-4,1)"]

len(nonfibered_names)

41

In [4]:
all([snappy.Manifold(name).homology().betti_number() == 1 for name in nonfibered_names])

True

They all have Alexander norm $4$:

In [5]:
all([snappy.Manifold(name).alexander_polynomial().degree() == 4 for name in nonfibered_names])

True

Now we check that they have Thurston norm $2$, by showing that each of these manifolds contains a normal surface with $\chi = -2$.\
This optimal value is attained even if we only look at the fundamental normal surfaces.

In [12]:
def has_chi_minus2_surface(name):
    tri = regina.Triangulation3(snappy.Manifold(name).filled_triangulation())
    ns = regina.NormalSurfaces(tri, regina.NormalCoords.NS_STANDARD)
    return any(surf.eulerChar() == -2 for surf in ns
                  if surf.isOrientable() and surf.cutAlong().countComponents() == 1 and surf.isConnected())

In [13]:
%time all(has_chi_minus2_surface(name) for name in nonfibered_names)

CPU times: user 30.8 s, sys: 20.2 ms, total: 30.8 s
Wall time: 30.8 s


True

## The algorithm

We can test our algorithm on all the census manifolds with $b_1=1$, now that we know their Thurston norms:

    Thurston norm = Alexander norm - 2

We cannot find large finite quotients (likely because $\mathrm{rk}(G) = 1$), but even the trivial quotient gives the correct result!

In [8]:
mfds = [snappy.Manifold(name) for name in nonfibered_names]

In [15]:
twisted_l2.load_hap()

true

In [14]:
def make_chain_complex(mfd_name):
    tri = regina.Triangulation3(snappy.Manifold(mfd_name).filled_triangulation())
    fl = twisted_l2.regina_tri_to_face_lattice(tri, ideal=False, simplify=True)
    cw = twisted_l2.cw_complex(fl)
    chain_complex = twisted_l2.equivariant_cc(cw)
    G = twisted_l2.get_fundamental_group(chain_complex)
    cc = twisted_l2.get_differentials(chain_complex)
    return G, cc

In [41]:
def stabilize_valuations(G, cc, exps, tolerance = 0.25):
    lvl = twisted_l2.configs.LogOptions.LEVEL
    twisted_l2.configs.LogOptions.LEVEL = twisted_l2.SILENT
    
    n = 1
    vals = [tolerance+1]*(len(cc)+1)
    vals2 = [0]*(len(cc)+1)
    ret_val = []
    while any(abs(a-b) > tolerance for a,b in zip(vals, vals2)):
        print(f"n = {n : >2}: ", end = "")
        ret = twisted_l2.characteristic(G, [1], cc, n, exps)
        ret_val.append(ret)
        vals2 = vals
        vals = [l.valuation for l in twisted_l2.get_twisted_l2_logs()]
        n += 1
        print(f"{float(ret):>9.5f} (valuations: {[float(vv) for vv in vals]})")

    twisted_l2.configs.LogOptions.LEVEL = lvl
    return ret_val

In [17]:
%time G,cc = make_chain_complex(nonfibered_names[0])

CPU times: user 142 ms, sys: 3.34 ms, total: 145 ms
Wall time: 145 ms


In [39]:
%time stabilize_valuations(G, cc, ())

n =  1: -8.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  2: -6.00000 (valuations: [0.0, 4.0, 4.0, 0.0])
n =  3: -4.00000 (valuations: [0.0, 6.0, 6.0, 0.0])
n =  4: -4.00000 (valuations: [0.0, 8.0, 7.0, 0.0])
n =  5: -2.00000 (valuations: [0.0, 8.0, 8.0, 0.0])
n =  6: -2.00000 (valuations: [0.0, 8.0, 8.0, 0.0])
CPU times: user 873 ms, sys: 51 µs, total: 874 ms
Wall time: 875 ms


[-8, -6, -4, -4, -2, -2]

In [43]:
%%time
characteristics = []
for name in nonfibered_names:
    print(f"===== {name} =====")
    G, cc = make_chain_complex(name)
    char_approx = stabilize_valuations(G, cc, ())
    characteristics.append(char_approx[-1])
    print(f"Characteristic: {char_approx[-1]}")
    print()

===== s528(-1,3) =====
n =  1:  -4.00000 (valuations: [0.0, 2.0, 3.0, 0.0])
n =  2:  -2.00000 (valuations: [0.0, 4.0, 5.0, 0.0])
n =  3:  -1.00000 (valuations: [0.0, 5.0, 6.0, 0.0])
n =  4:  -2.00000 (valuations: [0.0, 6.0, 6.0, 0.0])
n =  5:  -2.00000 (valuations: [0.0, 6.0, 6.0, 0.0])
Characteristic: -2

===== s527(-5,1) =====
n =  1:  -3.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  2:  -3.00000 (valuations: [0.0, 4.0, 3.0, 0.0])
n =  3:  -2.00000 (valuations: [0.0, 5.0, 4.0, 0.0])
n =  4:  -2.00000 (valuations: [0.0, 5.0, 4.0, 0.0])
Characteristic: -2

===== s644(-4,3) =====
n =  1:  -5.00000 (valuations: [0.0, 1.0, 1.0, 0.0])
n =  2:  -4.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  3:  -3.00000 (valuations: [0.0, 3.0, 3.0, 0.0])
n =  4:  -2.00000 (valuations: [0.0, 4.0, 4.0, 0.0])
n =  5:  -2.00000 (valuations: [0.0, 4.0, 4.0, 0.0])
Characteristic: -2

===== s643(-5,1) =====
n =  1:  -5.00000 (valuations: [0.0, 1.0, 1.0, 0.0])
n =  2:  -4.00000 (valuations: [0.0, 2.0, 2.0, 0.0])


This test is highly non-deterministic (because of the `filled_triangulation` method), so repeated runs may show different results.\
We expect the computed values of $\chi^{(2)}(\tilde M; \phi)$ (saved in `characteristics`) to be all equal to $-2$:

In [44]:
all(c == -2 for c in characteristics)

True

Finally, we check the $86$ fibered manifolds.

In [51]:
set_nonfibered = set(nonfibered_names)
fibered_names = [str(mfd) for mfd in snappy.OrientableClosedCensus(betti=1) if str(mfd) not in set_nonfibered]

for t in itertools.batched(fibered_names, 5):
    print("\t".join(t))

m160(3,1)	m159(4,1)	m199(-4,1)	m122(-4,1)	s942(-2,1)
m336(-1,3)	m345(1,2)	m289(7,1)	m280(1,4)	m304(-5,1)
m305(-1,3)	s385(5,1)	s296(-1,3)	s297(5,1)	s912(0,1)
m401(-2,3)	m371(-1,3)	m368(-4,1)	s580(-5,1)	s581(-1,3)
s869(-1,2)	s861(3,1)	v1191(-5,1)	v1076(-5,1)	s924(3,1)
v1408(4,1)	s677(1,3)	s676(5,1)	v2641(-4,1)	s745(3,2)
s646(5,2)	s789(-5,1)	s719(7,1)	v1373(-2,3)	v3209(3,1)
v2420(-3,1)	v2099(-4,1)	v2101(3,1)	s789(5,1)	v1539(-5,1)
v1721(1,4)	v2771(-4,1)	s836(-6,1)	v2986(1,2)	v2209(2,3)
v2054(-7,1)	v3066(-1,2)	v2563(5,1)	v2345(5,1)	v3209(-3,1)
v3077(5,1)	v2959(-3,1)	v2671(-2,3)	s928(2,3)	v3390(3,1)
v3209(4,1)	v2913(-3,2)	v3505(-3,1)	v3261(4,1)	v3262(3,1)
v2678(-5,1)	v3107(3,2)	v3216(4,1)	v3217(-1,3)	v3320(4,1)
v3091(-2,3)	v3214(1,3)	v3215(-4,1)	v3209(-4,1)	v2984(-1,3)
v3209(5,1)	v3019(5,2)	v3212(1,3)	v3209(-5,1)	v3425(-3,2)
v3209(6,1)	v3318(4,1)	v3352(1,4)	v3398(2,3)	v3378(-1,4)
v3408(1,3)	v3467(-2,3)	v3445(6,1)	v3509(4,3)	v3508(4,1)
v3504(-2,3)


In [52]:
%%time
success = True
for name in fibered_names:
    print(f"===== {name} =====")
    alex = snappy.Manifold(name).alexander_polynomial().degree()
    G, cc = make_chain_complex(name)
    char_approx = stabilize_valuations(G, cc, ())
    if char_approx[-1] != 2 - alex:
        success = False
    print(f"Characteristic: {char_approx[-1]}, expected: {2 - alex}")
    print()

===== m160(3,1) =====
n =  1:  -6.00000 (valuations: [0.0, 1.0, 1.0, 0.0])
n =  2:  -5.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  3:  -4.00000 (valuations: [0.0, 3.0, 3.0, 0.0])
n =  4:  -3.00000 (valuations: [0.0, 4.0, 4.0, 0.0])
n =  5:  -2.00000 (valuations: [0.0, 5.0, 5.0, 0.0])
n =  6:  -2.00000 (valuations: [0.0, 5.0, 5.0, 0.0])
Characteristic: -2, expected: -2

===== m159(4,1) =====
n =  1:  -1.00000 (valuations: [0.0, 1.0, 2.0, 0.0])
n =  2:  -2.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  3:  -2.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
Characteristic: -2, expected: -2

===== m199(-4,1) =====
n =  1:  -3.00000 (valuations: [0.0, 1.0, 1.0, 0.0])
n =  2:  -2.00000 (valuations: [0.0, 2.0, 2.0, 0.0])
n =  3:  -1.00000 (valuations: [0.0, 3.0, 3.0, 0.0])
n =  4:  -2.00000 (valuations: [0.0, 4.0, 3.0, 0.0])
n =  5:  -2.00000 (valuations: [0.0, 4.0, 3.0, 0.0])
Characteristic: -2, expected: -2

===== m122(-4,1) =====
n =  1:  -9.00000 (valuations: [0.0, 2.0, 1.0, 0.0])
n =  2:  -9.0

In [53]:
success

True

Finally, here is a table of all the manifolds and their twisted $L^2$-Euler characteristics (computed as $2-\lVert\cdot \rVert_A$).

In [59]:
for mfd in snappy.OrientableClosedCensus(betti=1):
    char = 2 - mfd.alexander_polynomial().degree()
    print(f"{str(mfd) :<11} : {char :>3}")

m160(3,1)   :  -2
m159(4,1)   :  -2
m199(-4,1)  :  -2
m122(-4,1)  :  -2
s942(-2,1)  :  -2
m336(-1,3)  :  -2
m345(1,2)   :  -4
m289(7,1)   :  -2
m280(1,4)   :  -2
m304(-5,1)  :  -2
m305(-1,3)  :  -2
s385(5,1)   :  -4
s296(-1,3)  :  -2
s297(5,1)   :  -2
s912(0,1)   :  -2
m401(-2,3)  :  -2
m371(-1,3)  :  -2
m368(-4,1)  :  -2
s580(-5,1)  :  -2
s581(-1,3)  :  -2
s869(-1,2)  :  -2
s861(3,1)   :  -2
v1191(-5,1) :  -2
v1076(-5,1) :  -2
s528(-1,3)  :  -2
s527(-5,1)  :  -2
s924(3,1)   :  -2
v1408(4,1)  :  -2
s677(1,3)   :  -2
s676(5,1)   :  -2
v2641(-4,1) :  -2
s745(3,2)   :  -2
s644(-4,3)  :  -2
s643(-5,1)  :  -2
s646(5,2)   :  -4
s789(-5,1)  :  -2
s719(7,1)   :  -2
v1373(-2,3) :  -2
v2018(-4,1) :  -2
v3209(3,1)  :  -2
v2420(-3,1) :  -2
v2099(-4,1) :  -2
v2101(3,1)  :  -2
s789(5,1)   :  -2
v1539(-5,1) :  -2
v1436(-5,1) :  -2
v1721(1,4)  :  -8
s750(4,3)   :  -2
s749(5,1)   :  -2
s789(-5,2)  :  -2
v1539(5,2)  :  -2
v2238(-5,1) :  -2
v3209(1,2)  :  -2
s828(-4,3)  :  -2
v1695(5,1)  :  -2
v2771(-4,1